In [14]:
from sage.symbolic.integration.integral import integrate

from sage.symbolic.integration.integral import definite_integral
# Anillo de polinomios de dos variables
R.<x,y> = PolynomialRing(QQ, order='degrevlex')

# Parámetros
alpha0, alpha1, alpha2 = 0, 1/2, 5 # Teoricamente las diferencias no pueden ser enteras
beta = 3
gamma = 2

# Producto escalar y funcional 0
dot_0 = lambda f, g: definite_integral(definite_integral(f*g*(x+1)**alpha0*y**beta*(-x-y)**gamma, y, 0, -x), \
    x, -1, 0)
L_0 = lambda P : dot_0(P,1)

def dotP_0(P, Q):
    M = P * transpose(Q)
    return matrix([[L_0(M[i,j]) for i in range(P.nrows())] for j in range(Q.nrows())])

# Producto escalar y funcional 1
dot_1 = lambda f, g: definite_integral(definite_integral(f*g*(x)**alpha1*y**beta*(1-x-y)**gamma, y, 0, 1-x), \
    x, 0, 1)
L_1 = lambda P : dot_1(P,1)

def dotP_1(P, Q):
    M = P * transpose(Q)
    return matrix([[L_1(M[i,j]) for i in range(P.nrows())] for j in range(Q.nrows())])

# Producto escalar y funcional 2
dot_2 = lambda f, g: definite_integral(definite_integral(f*g*(x-1)**alpha2*y**beta*(2-x-y)**gamma, y, 0, 2-x), \
    x, 1, 2)
L_2 = lambda P : dot_2(P,1)

def dotP_2(P, Q):
    M = P * transpose(Q)
    return matrix([[L_2(M[i,j]) for i in range(P.nrows())] for j in range(Q.nrows())])


# Polinomios ortogonales múltiples

Vamos a intentar crear polinomios ortogonales múltiples tipo Jacobi-Angelesco. Es decir, respecto a las funciones peso
$$
W_1(x,y) = (x+1)^{\alpha_1} y^\beta(-x-y)^\gamma\text{ en la región } \Omega_1 = \{(x,y)\in\mathbb R^2: x\geq -1, y\geq 0, y+x\geq 0 \},
$$

$$
W_2(x,y) = x^{\alpha_2} y^\beta(1-x-y)^\gamma\text{ en la región } \Omega_2 = \{(x,y)\in\mathbb R^2: x\geq 0, y\geq 0, y+x\geq 1 \},
$$

$$
W_3(x,y) = (x-1)^{\alpha_3} y^\beta(2-x-y)^\gamma\text{ en la región } \Omega_3 = \{(x,y)\in\mathbb R^2: x\geq 1, y\geq 0, y+x\geq 2 \}
$$

Las funciones peso son tipo Jacobi, y los interiores de cada región son disjuntos dos a dos (Angelesco).

Pensemos en alguna terna que verifique la condición $n(n+1)=\sum_{i=1}^3 n_i(n_i+1)$, por ejemplo: $(1, 3, 2)$ (grado 4). Calculamos el número de incógnitas

In [2]:
from math import comb
n_monomios = lambda n, d: comb(n + d - 1, n)
n_incognitas = lambda n, d : n_monomios(n, d) * sum([n_monomios(k,d) for k in range(n)])
n = 4
nin = n_incognitas(4, 2)
nin

50

In [3]:
n_ = (1,3,2)

Tenemos que
$$
P_{(1,4,4)} = \mathbb X_6 + \sum_{i=0}^5 G_{6,i} \mathbb X_i
$$

In [4]:
def a(i):
    return SR.var('a{}'.format(i))
incognitas = [a(i) for i in range(1, nin+1)]

In [5]:
G = []
contador = 0
for i in range(n):
    filas = []
    for j in range(n+1):
        fila = [a(contador+i) for i in range(i+1)]
        contador += (i+1)
        filas += [fila]
    G += [matrix(filas)]
G

[
[a0]  [ a5  a6]  [a15 a16 a17]  [a30 a31 a32 a33]
[a1]  [ a7  a8]  [a18 a19 a20]  [a34 a35 a36 a37]
[a2]  [ a9 a10]  [a21 a22 a23]  [a38 a39 a40 a41]
[a3]  [a11 a12]  [a24 a25 a26]  [a42 a43 a44 a45]
[a4], [a13 a14], [a27 a28 a29], [a46 a47 a48 a49]
]

In [6]:
monomios_grado = lambda n : [R(x**(n-j) * y**j) for j in range(n+1)]
X = [transpose(matrix(monomios_grado(j))) for j in range(n+1)]
X

[
                          [    x^4]
                 [  x^3]  [  x^3*y]
          [x^2]  [x^2*y]  [x^2*y^2]
     [x]  [x*y]  [x*y^2]  [  x*y^3]
[1], [y], [y^2], [  y^3], [    y^4]
]

In [7]:
Pn = X[n] + sum([G[i]*X[i] for i in range(n)])
Pn = matrix(Pn)
Pn

[     a30*x^3 + x^4 + a31*x^2*y + a32*x*y^2 + a33*y^3 + a15*x^2 + a16*x*y + a17*y^2 + a5*x + a6*y + a0]
[   a34*x^3 + a35*x^2*y + x^3*y + a36*x*y^2 + a37*y^3 + a18*x^2 + a19*x*y + a20*y^2 + a7*x + a8*y + a1]
[a38*x^3 + a39*x^2*y + a40*x*y^2 + x^2*y^2 + a41*y^3 + a21*x^2 + a22*x*y + a23*y^2 + a9*x + a10*y + a2]
[ a42*x^3 + a43*x^2*y + a44*x*y^2 + a45*y^3 + x*y^3 + a24*x^2 + a25*x*y + a26*y^2 + a11*x + a12*y + a3]
[   a46*x^3 + a47*x^2*y + a48*x*y^2 + a49*y^3 + y^4 + a27*x^2 + a28*x*y + a29*y^2 + a13*x + a14*y + a4]

In [8]:
ecs = flatten([dotP_0(Pn, X[i]).list() for i in range(n_[0])]) + \
flatten([dotP_1(Pn, X[i]).list() for i in range(n_[1])]) + \
flatten([dotP_2(Pn, X[i]).list() for i in range(n_[2])])
len(ecs)


NameError: name 'definite_integral' is not defined

In [ ]:
len(ecs) == nin

In [ ]:
ecs

In [ ]:
sol = solve(ecs, incognitas)
sol

Si las potencias son todas enteras, el sistema no tiene solución, algo que ya pasaba con el caso de una variable

In [ ]:
def mypow(a,b, gr = 100):
    loga = log(a)
    return sum([(b*loga)**n/factorial(n) for n in range(gr)])

1/2

In [23]:
alpha1 = 13
L_1(1)

1/32558400

In [ ]:
5**4.3498375

In [ ]:
mypow(5,4.3498375)

Situación actual: 
* Esto puede funcionar o no
* De momento sé que si los $\alpha_i$ son todos enteros el sistema no tiene solución (eso es bueno). Nos vale como contraejemplo para comprobar que no todas las ternas son normales cuando trabajamos en varias variables. Podría coger un ejemplo sencillo y desarrollarlo.
* El código strugglea cuando le meto potencias no enteras
* Intento hacerlo con integración numérica pero tampoco fufa.

Idea: Directamente programar mi propia integración numérica o intentar hacerlo con Mathematica